<a href="https://colab.research.google.com/github/mille055/Rec_Project/blob/main/notebooks/getting_more_podcast_reviews2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install app_store_scraper unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 KB 7.9 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.15
    Uninstalling urllib3-1.26.15:
      Successfully uninstalled urllib3-1.26.15
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: requests
    Found

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from app_store_scraper import AppStore
import json
import re
import unidecode
import time
from tqdm import tqdm
import pickle

In [3]:

!git clone 'https://github.com/mille055/Rec_Project.git'

Cloning into 'Rec_Project'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 202 (delta 39), reused 43 (delta 10), pack-reused 112
Receiving objects: 100% (202/202), 56.69 MiB | 14.84 MiB/s, done.
Resolving deltas: 100% (96/96), done.
Updating files: 100% (23/23), done.


In [6]:
myscraped = pd.read_pickle('/content/Rec_Project/data/archive/cmm_scraped_podcasts_with_ratings.pkl')
myscraped

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,itunes_id,episode_descriptions,user,rating
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,RobinFerris,5.0
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,Pops.99,1.0
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,ReddEye81,5.0
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,Keyta7777,2.0
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,Okkupent,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,Monijansand,5.0
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,trinityangel13,5.0
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,Kweenkeys,5.0
4036,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,JoyfulJoyfulWOG,5.0


In [7]:
set_of_links = set(myscraped.link.tolist())
set_of_links

{'https://podcasts.apple.com/us/podcast/axe-of-the-blood-god-an-rpg-podcast/id969659784',
 'https://podcasts.apple.com/us/podcast/perfecto-podcast-featuring-paul-oakenfold/id307580286',
 'https://podcasts.apple.com/us/podcast/money-guy-show/id121362031',
 'https://podcasts.apple.com/us/podcast/americas-town-hall-with-heidi-st-john/id1565223008',
 'https://podcasts.apple.com/us/podcast/china-in-the-world/id738323123',
 'https://podcasts.apple.com/us/podcast/bedtime-explorers/id1333863338',
 'https://podcasts.apple.com/us/podcast/lore/id978052928',
 'https://podcasts.apple.com/us/podcast/calm-pills-soothing-space-ambient-and-piano-music-for/id1173559723',
 'https://podcasts.apple.com/us/podcast/conan-obrien-needs-a-friend/id1438054347',
 'https://podcasts.apple.com/us/podcast/lions-led-by-donkeys-podcast/id1393845532',
 'https://podcasts.apple.com/us/podcast/battleline-podcast/id1486583326',
 'https://podcasts.apple.com/us/podcast/empowerher/id1444456380',
 'https://podcasts.apple.com/us

In [8]:
country = "us"
num_pages = 10  # Adjust this to fetch more or fewer pages of reviews

In [9]:
# def fetch_individual_ratings(link, country = 'us', num_pages=10):
#     app_name = get_app_name(link)
#     print('app name is ', app_name)
#     podcast_id = get_podcast_id(link)
#     print('podcast_id is', podcast_id)
#     podcast = AppStore(country=country, app_name = app_name, app_id=podcast_id)
#     podcast.review(how_many=num_pages)
#     reviews = podcast.reviews
#     individual_ratings = []

#     for review in reviews:
#         individual_ratings.append({
#             'itunes_id': podcast_id,
#             'user': review['userName'],
#             'rating': review['rating'],
#             'title': review['title'],
#             'review': review['review']
#         })

#     return individual_ratings


# def get_app_name(url):
#     app_name = url.split("/id")[-2]
#     return app_name


# def get_podcast_id(url):
#     podcast_id = url.split("/id")[-1].split("?")[0]
#     return podcast_id

In [ ]:
# test = ['https://podcasts.apple.com/us/podcast/hbos-the-last-of-us-podcast/id1660320068',
#  'https://podcasts.apple.com/us/podcast/this-is-uncomfortable/id1466681015',
#  'https://podcasts.apple.com/us/podcast/thomas-friends-storytime-us/id1506815544',
#  'https://podcasts.apple.com/us/podcast/ali-on-the-run-show/id1204523870']

In [10]:
# ratings = []
# for item in set_of_links:
#   rating = fetch_individual_ratings(item)
#   ratings.append(rating)

In [ ]:
len(ratings)

In [ ]:
len(ratings[0])

In [ ]:
len(ratings[1])

12

In [ ]:
clean_data = [item for item in ratings if item]
len(clean_data)

596

In [ ]:
flattened_data = [item for sublist in clean_data for item in sublist]
len(flattened_data)

10689

,itunes_id,user,rating,title,review
0,1436541194,And then Jill,5,Experiencing Awe,Great podcast in general. This weeks interview...
1,1436541194,germaine de stael,5,Pretty amazing,There is nothing slow or boring about this pod...
2,1436541194,Maida.K,5,"Interesting, Informative and Inspiring.",Thank you for creating this podcast! Just what...
3,1436541194,mituska2,5,Fascinating topics,This podcast interviews scientists and experts...
4,1436541194,DW39A,5,WONDERFUL!!!!!,This show helps re-create the wonder I had as ...
...,...,...,...,...,...
10684,1642377102,Jra4776,5,⭐️⭐️⭐️⭐️⭐️,Absolutely love the dead files. 5⭐️’s. Definit...
10685,1642377102,horrorfanbob,5,Unique approach,I really like the way Amy and Steve approach p...
10686,1642377102,Mulder19,5,Love it- please add more episodes!,My favorite show - love it and it translates s...
10687,1642377102,whynotpugs,5,One of my favorite Travel Channel Fixes!,There are so many seasons and episodes that ar...


In [ ]:
ratings_df.user.value_counts()

obacker19        8
EricaBushwell    7
Suzannekmoses    7
A UK fan         6
ASobering        6
                ..
Pa$:63dhi        1
Cece LA          1
KidCoodieSick    1
Tim Woosley      1
paranorwhore     1
Name: user, Length: 10364, dtype: int64

In [ ]:
ratings_df.to_pickle('firstscrape_ratings.pkl')

In [11]:
## trying again, this time with delays to help not run afowl of the site

!pip install tenacity


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from tenacity import retry, wait_fixed, stop_after_attempt
import time

In [13]:
@retry(wait=wait_fixed(5), stop=stop_after_attempt(5))
def fetch_individual_ratings(link, country = 'us', num_pages=10):
    app_name = get_app_name(link)
    print('app name is ', app_name)
    podcast_id = get_podcast_id(link)
    print('podcast_id is', podcast_id)
    podcast = AppStore(country=country, app_name = app_name, app_id=podcast_id)
    podcast.review(how_many=num_pages)
    reviews = podcast.reviews
    individual_ratings = []

    for review in reviews:
        individual_ratings.append({
            'itunes_id': podcast_id,
            'user': review['userName'],
            'rating': review['rating'],
            'title': review['title'],
            'review': review['review']
        })

    return individual_ratings


def get_app_name(url):
    app_name = url.split("/id")[-2]
    return app_name


def get_podcast_id(url):
    podcast_id = url.split("/id")[-1].split("?")[0]
    return podcast_id

In [14]:
def divide_list_into_chunks(input_list, chunk_size):
    return [input_list[i:i + chunk_size] for i in range(0, len(input_list), chunk_size)]


In [15]:
batch_size = 100  # Adjust this based on your requirements
batches = divide_list_into_chunks(list(set_of_links), batch_size)

In [35]:
ratings = []
batch_counter = 0
item_counter = 0
for batch in batches[13:]:
  print('On batch', batch_counter)
  ratings_batch = []
  for item in batch:
    print('getting item', item_counter)
    rating = fetch_individual_ratings(item)
    ratings_batch.append(rating)
    item_counter=item_counter+1
  batch_counter=batch_counter+1
  ratings.append(ratings_batch)
  

On batch 0
getting item 0
app name is  https://podcasts.apple.com/us/podcast/energy-policy-now
podcast_id is 1191681161


ERROR:Base:Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/us/apps/1191681161/reviews?l=en-GB&offset=0&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 429 error responses'))


getting item 1
app name is  https://podcasts.apple.com/us/podcast/andrew-schulzs-flagrant-with-akaash-singh
podcast_id is 1643946763


ERROR:Base:Something went wrong: HTTPSConnectionPool(host='amp-api.apps.apple.com', port=443): Max retries exceeded with url: /v1/catalog/us/apps/1643946763/reviews?l=en-GB&offset=0&limit=20&platform=web&additionalPlatforms=appletv%2Cipad%2Ciphone%2Cmac (Caused by ResponseError('too many 429 error responses'))


getting item 2
app name is  https://podcasts.apple.com/us/podcast/john-solomon-reports
podcast_id is 1495251160


...

ERROR:Base:Keyboard interrupted


getting item 34
app name is  https://podcasts.apple.com/us/podcast/the-michael-steele-podcast
podcast_id is 1412905534


ERROR:Base:Keyboard interrupted


getting item 35


KeyboardInterrupt: ignored

In [19]:
len(ratings)

3

In [20]:
clean_data = [item for item in ratings if item]
len(clean_data)

3

In [26]:
ratings[2]

[[{'itunes_id': '1671087656',
   'user': 'Jon-O-pan',
   'rating': 5,
   'title': 'Informed and fun',
   'review': 'Love listening to them explore ideas about the future.'},
  {'itunes_id': '1671087656',
   'user': 'Bgomski',
   'rating': 2,
   'title': 'Complainers',
   'review': 'I listened to the first part of an episode and instead of talking about the benefits of what’s coming they just complain about everything not going the way they think it should.'},
  {'itunes_id': '1671087656',
   'user': 'Pete-r',
   'rating': 5,
   'title': 'Loving it!',
   'review': 'Great round up of of the current trends and issues in AI.'},
  {'itunes_id': '1671087656',
   'user': 'Newshoes522',
   'rating': 5,
   'title': 'Insightful and fascinating!',
   'review': 'I was searching for podcasts with up to date stories and intelligent analysis of the latest developments in AI and just came across this gem. Loved the first two episodes and especially appreciate the detailed show notes! Look forward to h

In [27]:
flattened_ratings = [item for sublist in ratings for item in sublist]
len(flattened_ratings)

300

In [30]:
flattened_ratings_second = [item for sublist in flattened_ratings for item in sublist]

In [31]:
len(flattened_ratings_second)

5167

In [32]:
ratings1000_1300_df = pd.DataFrame(flattened_ratings_second)

In [33]:
ratings1000_1300_df

,itunes_id,user,rating,title,review
0,1546000046,LitLateLime,5,where are the March stories?!,I really hope you didn’t give up on this podca...
1,1546000046,catcatgattokatze,5,This is great,Good
2,1546000046,Idk what to called me,5,My favorite podcast ever,First before I write why I love this dude plea...
3,1546000046,Broarmen,5,Confused,Anybody who’s telling him to do his thing diff...
4,1546000046,PrecociousWeirdo,5,Totally Top 3!,Hiya Dark! This is in my top 3 creepypasta pod...
...,...,...,...,...,...
5162,1637476174,oregonbuckeye339,5,Hooked!,Listened to my first episode yesterday and imm...
5163,1637476174,robbiedorius,5,feels like i’m chatting with my friends,i absolutely love this show and the three geni...
5164,1637476174,Special Dark w/ Almonds,5,I didn’t see this coming!,I heard about this podcast from Matt Bellassai...
5165,1637476174,rocketrocks,5,This podcast is a gift,This podcast means so much to me. I love liste...


In [34]:
ratings1000_1300_df.to_pickle('ratings1000_1300.pkl')

In [36]:
scraped_previously = pd.read_pickle('/content/Rec_Project/data/cmm_scraped2.pkl')

In [37]:
scraped_previously

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,itunes_id,episode_descriptions,rating,user
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,5,RobinFerris
1,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,1,Pops.99
2,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,5,ReddEye81
3,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,2,Keyta7777
4,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,4,Okkupent
...,...,...,...,...,...,...,...,...,...,...,...,...
440915,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,5,Monijansand
440916,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,5,trinityangel13
440917,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,5,Kweenkeys
440918,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,5,JoyfulJoyfulWOG


In [38]:
scraped_previously = scraped_previously.drop_duplicates(subset=['itunes_id', 'rating', 'user'])
scraped_previously

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,itunes_id,episode_descriptions,rating,user
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,5,RobinFerris
1,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,1,Pops.99
2,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,5,ReddEye81
3,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,2,Keyta7777
4,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,1526579247,[In celebration of our new premium format—two ...,4,Okkupent
...,...,...,...,...,...,...,...,...,...,...,...,...
440825,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,5,Monijansand
440826,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,5,trinityangel13
440827,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,5,Kweenkeys
440828,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,1512702672,[Hey Angels!!! In today's episode of the Quali...,5,JoyfulJoyfulWOG


In [39]:
scraped_previously.columns

Index(['title', 'producer', 'genre', 'description', 'num_episodes',
       'avg_rating', 'num_reviews', 'link', 'itunes_id',
       'episode_descriptions', 'rating', 'user'],
      dtype='object')

In [41]:
base_for_new_batch = scraped_previously[['title', 'producer', 'genre', 'description', 'num_episodes', 'avg_rating', 'num_reviews', 'link', 'episode_descriptions', 'itunes_id']]
base_for_new_batch

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,episode_descriptions,itunes_id
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247
1,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247
2,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247
3,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247
4,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247
...,...,...,...,...,...,...,...,...,...,...
440825,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672
440826,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672
440827,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672
440828,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672


In [47]:
old_batch = scraped_previously[['itunes_id', 'rating', 'user']]

In [46]:
new_batch = ratings1000_1300_df[['itunes_id', 'rating', 'user']]
new_batch

,itunes_id,rating,user
0,1546000046,5,LitLateLime
1,1546000046,5,catcatgattokatze
2,1546000046,5,Idk what to called me
3,1546000046,5,Broarmen
4,1546000046,5,PrecociousWeirdo
...,...,...,...
5162,1637476174,5,oregonbuckeye339
5163,1637476174,5,robbiedorius
5164,1637476174,5,Special Dark w/ Almonds
5165,1637476174,5,rocketrocks


In [48]:
old_batch

,itunes_id,rating,user
0,1526579247,5,RobinFerris
1,1526579247,1,Pops.99
2,1526579247,5,ReddEye81
3,1526579247,2,Keyta7777
4,1526579247,4,Okkupent
...,...,...,...
440825,1512702672,5,Monijansand
440826,1512702672,5,trinityangel13
440827,1512702672,5,Kweenkeys
440828,1512702672,5,JoyfulJoyfulWOG


In [52]:
combined = pd.concat((old_batch, new_batch), axis=0, ignore_index=True)
combined

,itunes_id,rating,user
0,1526579247,5,RobinFerris
1,1526579247,1,Pops.99
2,1526579247,5,ReddEye81
3,1526579247,2,Keyta7777
4,1526579247,4,Okkupent
...,...,...,...
50014,1637476174,5,oregonbuckeye339
50015,1637476174,5,robbiedorius
50016,1637476174,5,Special Dark w/ Almonds
50017,1637476174,5,rocketrocks


In [53]:
combined.to_pickle('all_ratings_so_far.pkl')

In [54]:
### re-combining the reviews with the base

In [55]:
latest_podcast_df = base_for_new_batch.merge(combined, on='itunes_id')
latest_podcast_df

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,episode_descriptions,itunes_id,rating,user
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,5,RobinFerris
1,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,1,Pops.99
2,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,5,ReddEye81
3,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,2,Keyta7777
4,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,4,Okkupent
...,...,...,...,...,...,...,...,...,...,...,...,...
657284,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,Monijansand
657285,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,trinityangel13
657286,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,Kweenkeys
657287,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,JoyfulJoyfulWOG


In [57]:
latest_podcast_df.user.value_counts()

Maxcraft430               377
obacker19                 370
malfoxley                 341
Suzannekmoses             291
Arlie K                   274
                         ... 
NavySteve87!                1
Ruth of Bedford County      1
PattrickO                   1
Nolikeynewudatey            1
gjsixhujd                   1
Name: user, Length: 42243, dtype: int64

In [58]:
latest_podcast_df[latest_podcast_df.user=='Maxcraft430']

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,episode_descriptions,itunes_id,rating,user
48225,Life Will Be the Death of Me with Chelsea Handler,iHeartPodcasts,Comedy,Listen to Chelsea Handler's first ever podcast...,117,4.3,4300.0,https://podcasts.apple.com/us/podcast/life-wil...,[Chelsea and Catherine are joined by Jen Since...,1462096941,5,Maxcraft430
48235,Life Will Be the Death of Me with Chelsea Handler,iHeartPodcasts,Comedy,Listen to Chelsea Handler's first ever podcast...,117,4.3,4300.0,https://podcasts.apple.com/us/podcast/life-wil...,[Chelsea and Catherine are joined by Jen Since...,1462096941,5,Maxcraft430
48255,Life Will Be the Death of Me with Chelsea Handler,iHeartPodcasts,Comedy,Listen to Chelsea Handler's first ever podcast...,117,4.3,4300.0,https://podcasts.apple.com/us/podcast/life-wil...,[Chelsea and Catherine are joined by Jen Since...,1462096941,5,Maxcraft430
48265,Life Will Be the Death of Me with Chelsea Handler,iHeartPodcasts,Comedy,Listen to Chelsea Handler's first ever podcast...,117,4.3,4300.0,https://podcasts.apple.com/us/podcast/life-wil...,[Chelsea and Catherine are joined by Jen Since...,1462096941,5,Maxcraft430
48285,Life Will Be the Death of Me with Chelsea Handler,iHeartPodcasts,Comedy,Listen to Chelsea Handler's first ever podcast...,117,4.3,4300.0,https://podcasts.apple.com/us/podcast/life-wil...,[Chelsea and Catherine are joined by Jen Since...,1462096941,5,Maxcraft430
...,...,...,...,...,...,...,...,...,...,...,...,...
639732,S-Town,Serial,Society & Culture,S-Town is a podcast hosted by Brian Reed from ...,8,4.6,44000.0,https://podcasts.apple.com/us/podcast/s-town/i...,[A mysterious letter detailing a supposed Isla...,1212558767,3,Maxcraft430
639752,S-Town,Serial,Society & Culture,S-Town is a podcast hosted by Brian Reed from ...,8,4.6,44000.0,https://podcasts.apple.com/us/podcast/s-town/i...,[A mysterious letter detailing a supposed Isla...,1212558767,3,Maxcraft430
639762,S-Town,Serial,Society & Culture,S-Town is a podcast hosted by Brian Reed from ...,8,4.6,44000.0,https://podcasts.apple.com/us/podcast/s-town/i...,[A mysterious letter detailing a supposed Isla...,1212558767,3,Maxcraft430
639782,S-Town,Serial,Society & Culture,S-Town is a podcast hosted by Brian Reed from ...,8,4.6,44000.0,https://podcasts.apple.com/us/podcast/s-town/i...,[A mysterious letter detailing a supposed Isla...,1212558767,3,Maxcraft430


In [60]:
latest_podcast_df = latest_podcast_df.drop_duplicates(subset=['itunes_id', 'user', 'rating'])
latest_podcast_df

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,episode_descriptions,itunes_id,rating,user
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,5,RobinFerris
1,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,1,Pops.99
2,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,5,ReddEye81
3,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,2,Keyta7777
4,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,4,Okkupent
...,...,...,...,...,...,...,...,...,...,...,...,...
657194,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,Monijansand
657195,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,trinityangel13
657196,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,Kweenkeys
657197,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,JoyfulJoyfulWOG


In [61]:
latest_podcast_df.to_pickle('podcast_df_040423.pkl')